# **Clean**

In [180]:
import pandas as pd

## Read Data

In [181]:
df_orders = pd.read_csv("raw/orders.csv")
df_restaurants = pd.read_csv("raw/restaurants.csv")

## Cleaning Columns

In [182]:
print("Orders Columns: \n", df_orders.columns)
print("Restaurants Columns: \n", df_restaurants.columns)

Orders Columns: 
 Index(['order_acknowledged_at', 'order_ready_at', 'order_value_gbp',
       'restaurant_id', 'number_of_items', 'prep_time_seconds'],
      dtype='object')
Restaurants Columns: 
 Index(['restaurant_id', 'country', 'city', 'type_of_food'], dtype='object')


### *df_orders.order_acknowledged_at*

In [183]:
df_orders.order_acknowledged_at

0        2015-06-01 12:28:28.952789+01:00
1        2015-06-06 17:06:24.434807+01:00
2        2015-06-08 14:56:15.503204+01:00
3        2015-06-12 15:12:20.497925+01:00
4        2015-06-01 13:07:10.794050+01:00
                       ...               
32389    2015-06-30 21:00:29.131078+01:00
32390    2015-06-30 22:30:51.123180+02:00
32391    2015-06-30 21:50:11.585362+01:00
32392    2015-06-30 22:17:55.778938+01:00
32393    2015-06-30 22:55:39.128579+01:00
Name: order_acknowledged_at, Length: 32394, dtype: object

The datetime column can be broken down to year, month, day of the week and hour of the day which may be good predictors.

### *df_orders.order_ready_at*

In [185]:
df_orders = df_orders.drop(columns=["order_ready_at"])

This column can not be used as it will not be available when predicting prep time.

### *df_orders.order_value_gbp*

In [186]:
df_orders.order_value_gbp.value_counts(dropna=False)

15.000000     549
15.500000     495
16.000000     466
17.000000     395
18.000000     321
             ... 
76.950000       1
139.450000      1
232.750000      1
83.400000       1
29.354365       1
Name: order_value_gbp, Length: 2845, dtype: int64

### *df_orders.prep_time_seconds*

In [187]:
print("99th percentile took", df_orders.prep_time_seconds.quantile(0.99) / 60 / 60, " hours.")
print("Longest delivery took ", df_orders.prep_time_seconds.max() / 60 / 60 / 24, " days.")
print(df_orders.prep_time_seconds.sort_values(ascending=False)[:100])
print(df_orders.iloc[:100].groupby("restaurant_id"))

99th percentile took 2.120613888888889  hours.
Longest delivery took  2.957164351851852  days.
9807     255499
31228    116038
13166    113157
31749    111836
20760    110198
          ...  
27102     21798
31224     21750
18337     21649
31323     21279
10978     20801
Name: prep_time_seconds, Length: 100, dtype: int64


Upper bounds of the prep time seems to be irregular. There may some special items such as cakes etc. that take multiple days to prepare. Will leave this for now but may examine filtering the upper quartile later or examining further the restaurants where long prep times are common.

In [188]:
# Looking further at the largest outlier
df_orders.iloc[9807]

order_acknowledged_at    2015-06-19 14:22:41.266706+01:00
order_value_gbp                                      76.8
restaurant_id                                         441
number_of_items                                         5
prep_time_seconds                                  255499
Name: 9807, dtype: object

### *df_restaurants.country*

In [189]:
df_restaurants.country.unique()

array(['UK', 'France', 'Ireland', 'Germany'], dtype=object)

### *df_restaurants.city*

In [190]:
df_restaurants.city.unique()

array(['London', 'Paris', 'Brighton', 'Manchester', 'Oxford', 'Cambridge',
       'Reading', 'Birmingham', 'Bristol', 'Dublin', 'Berlin',
       'Nottingham', 'Liverpool', 'Leeds', 'Edinburgh', 'Glasgow',
       'Guildford', 'München', 'Cheltenham', 'Leicester', 'Cardiff',
       'Southampton'], dtype=object)

In [191]:
num_country = pd.Series(pd.factorize(df_restaurants['country'])[0])
num_city = pd.Series(pd.factorize(df_restaurants['city'])[0])

num_country.corr(num_city)

0.3989143329851137

Here we can see only 1 city corresponding to Ireland, 2 cities corresponding to Germany and 1 city corresponding to France. 
This indicates that the country flag may not be very useful. It will also lead to city and country being correlated. 
For these reasons country will likely be dropped.

### *df_restaurants.type_of_food*

In [177]:
print(df_restaurants.type_of_food.unique())

['thai' 'italian' 'chinese' 'korean' 'lebanese' 'greek' 'japanese'
 'indian' 'mediterranean' 'american' 'burgers' 'healthy' 'vietnamese'
 'persian' 'burritos' 'argentinian' 'rotisserie' 'spanish tapas' 'dessert'
 'sichuan chinese' 'lebanese moroccan' 'sandwiches' 'juices' 'georgian'
 'pan asian' 'south indian' 'german' 'british' 'chicken' 'delicatessen'
 'european' 'austrian' 'peruvian' 'turkish' 'pizza' 'fish and chips'
 'mexican' 'australian' 'spanish' 'vegan' 'creole' 'ethiopian'
 'latin american' 'modern british' 'caribbean' 'bbq' 'ramen' 'soup'
 'international' 'portuguese' 'middle eastern' 'japanese peruvian'
 'bakery' 'brazilian' 'asian fusion' 'sweet treats' 'deli' 'french'
 'south african' 'south american' 'indonesian' 'bagels' 'malaysian'
 'nepalese' 'diner' 'north african' 'creperie' 'irish' 'gluten free'
 'bistro' 'french lebanese' 'sushi' 'sans gluten / brunch' 'thaï / brunch'
 'american / brunch' 'kosher' 'russian' 'taiwanese' 'seafood'
 'scandinavian' 'scottish' 'steakho

In [203]:
print(df_restaurants.type_of_food.value_counts().tail(25))

greek             15
korean            14
turkish           14
juices            13
bakery            10
caribbean          9
brazilian          8
sushi              8
german             7
latin american     7
north african      6
sandwiches         6
indonesian         5
rotisserie         5
vegan              5
australian         5
peruvian           5
gluten free        4
seafood            4
bagels             3
nepalese           3
bistro             3
irish              3
georgian           3
russian            3
Name: type_of_food, dtype: int64


In [202]:
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("ice cream", "dessert")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("delicatessen", "deli")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("soup", "international")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("portuguese", "international")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("japanese peruvian", "japanese")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("asian fusion", "pan asian")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("sweet treats", "dessert")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("south african", "international")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("south american", "peruvian")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("diner", "bistro")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("french lebanese", "lebanese")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("ethiopian", "international")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("steakhouse", "international")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("scottish", "european")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("sans gluten / brunch", "gluten free")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("scandinavian", "european")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("taiwanese", "chinese")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("kosher", "middle eastern")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("thaï / brunch", "thai")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("sichuan chinese", "chinese")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("spanish tapas", "mexican")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("creole", "international")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("ramen", "pan asian")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("malaysian", "indonesian")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("austrian", "european")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("creperie", "dessert")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("lebanese moroccan", "lebanese")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("modern british", "british")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("american / brunch", "american")
df_restaurants.type_of_food = df_restaurants.type_of_food.replace("south indian", "indian")


This column will need to be revisited once dataframes are merged to see if there are any food types which have few individual orders.

### *Correlation*

In [204]:
df_orders.corr()

,order_value_gbp,restaurant_id,number_of_items,prep_time_seconds
order_value_gbp,1.000000,-0.056161,0.537032,0.272160
restaurant_id,-0.056161,1.000000,-0.043202,0.009657
number_of_items,0.537032,-0.043202,1.000000,0.091153
prep_time_seconds,0.272160,0.009657,0.091153,1.000000


Above we can see that *order value* and *number of items* are **moderately** correlated. This is not definitavely a problem, however, it may be useful to create one feature which captures the information of both columns or drop one of the columns completely. I would be tentitive to drop either column as I feel both capture valuable information.

### Merge Data

In [205]:
df_orders

,order_acknowledged_at,order_value_gbp,restaurant_id,number_of_items,prep_time_seconds
0,2015-06-01 12:28:28.952789+01:00,59.90000,1326,2,6220
1,2015-06-06 17:06:24.434807+01:00,24.00000,1326,8,603
2,2015-06-08 14:56:15.503204+01:00,15.25000,1326,3,443
3,2015-06-12 15:12:20.497925+01:00,28.05000,1326,8,669
4,2015-06-01 13:07:10.794050+01:00,56.30000,255,7,3314
...,...,...,...,...,...
32389,2015-06-30 21:00:29.131078+01:00,35.15000,3185,4,913
32390,2015-06-30 22:30:51.123180+02:00,9.15771,3556,3,201
32391,2015-06-30 21:50:11.585362+01:00,18.20000,3517,3,863
32392,2015-06-30 22:17:55.778938+01:00,21.50000,1705,3,1339


In [206]:
df_restaurants

,restaurant_id,country,city,type_of_food
0,3,UK,London,thai
1,5,UK,London,italian
2,7,UK,London,italian
3,8,UK,London,chinese
4,9,France,Paris,korean
...,...,...,...,...
1692,3594,UK,Leeds,sushi
1693,3614,France,Paris,indonesian
1694,3618,Ireland,Dublin,pan asian
1695,3620,UK,Southampton,burritos


In [207]:
df_train = df_orders.merge(df_restaurants)
df_train

,order_acknowledged_at,order_value_gbp,restaurant_id,number_of_items,prep_time_seconds,country,city,type_of_food
0,2015-06-01 12:28:28.952789+01:00,59.90000,1326,2,6220,UK,London,burritos
1,2015-06-06 17:06:24.434807+01:00,24.00000,1326,8,603,UK,London,burritos
2,2015-06-08 14:56:15.503204+01:00,15.25000,1326,3,443,UK,London,burritos
3,2015-06-12 15:12:20.497925+01:00,28.05000,1326,8,669,UK,London,burritos
4,2015-06-01 13:07:10.794050+01:00,56.30000,255,7,3314,UK,London,chinese
...,...,...,...,...,...,...,...,...
32389,2015-06-30 21:00:29.131078+01:00,35.15000,3185,4,913,UK,Glasgow,italian
32390,2015-06-30 22:30:51.123180+02:00,9.15771,3556,3,201,Germany,Berlin,vietnamese
32391,2015-06-30 21:50:11.585362+01:00,18.20000,3517,3,863,UK,Cheltenham,indian
32392,2015-06-30 22:17:55.778938+01:00,21.50000,1705,3,1339,UK,London,italian


In [215]:
df_train.type_of_food.value_counts().tail(25)

pan asian         933
british           882
pizza             806
mexican           653
burritos          608
french            604
dessert           430
korean            387
turkish           360
greek             334
deli              313
argentinian       311
mediterranean     309
fish and chips    282
persian           270
international     246
european          229
bbq               227
south american    221
middle eastern    193
seafood           136
australian        108
spanish           101
bakery             69
juices             64
Name: type_of_food, dtype: int64

In [214]:
df_train.type_of_food = df_train.type_of_food.replace("irish", "european")
df_train.type_of_food = df_train.type_of_food.replace("russian", "international")
df_train.type_of_food = df_train.type_of_food.replace("bagels", "deli")
df_train.type_of_food = df_train.type_of_food.replace("sandwiches", "deli")
df_train.type_of_food = df_train.type_of_food.replace("gluten free", "healthy")
df_train.type_of_food = df_train.type_of_food.replace("georgian", "international")
df_train.type_of_food = df_train.type_of_food.replace("bistro", "international")
df_train.type_of_food = df_train.type_of_food.replace("north african", "middle eastern")
df_train.type_of_food = df_train.type_of_food.replace("nepalese", "indian")
df_train.type_of_food = df_train.type_of_food.replace("german", "european")
df_train.type_of_food = df_train.type_of_food.replace("vegan", "healthy")
df_train.type_of_food = df_train.type_of_food.replace("indonesian", "thai")
df_train.type_of_food = df_train.type_of_food.replace("sushi", "seafood")
df_train.type_of_food = df_train.type_of_food.replace("latin american", "mexican")
df_train.type_of_food = df_train.type_of_food.replace("rotisserie", "chicken")
df_train.type_of_food = df_train.type_of_food.replace("brazilian", "south american")
df_train.type_of_food = df_train.type_of_food.replace("peruvian", "south american")
df_train.type_of_food = df_train.type_of_food.replace("caribbean", "international")


In [217]:
df_train.to_csv("data/clean_data.csv", index=False)